# Advance Lane Lines

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

### Import useful packages

In [130]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import cv2
import pickle
%matplotlib inline

## Configuration

In [131]:
nx = 9
ny = 6
points_file = "camera_cal/points.p"

## Calibrating Camera

In [132]:

objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img_size = (img.shape[1], img.shape[0])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    print('processing image ' + fname)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        write_name = 'camera_cal/corners_found'+str(idx)+'.jpg'
        cv2.imwrite(write_name, img)
    else:
        print('No corner found')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)    
calibrated_data = {'mtx': mtx, 'dist': dist}
with open(points_file, 'wb') as f:
    pickle.dump(calibrated_data, file=f)
print("Prcoess camera cal image done")


processing image camera_cal/calibration1.jpg
No corner found
processing image camera_cal/calibration10.jpg
processing image camera_cal/calibration11.jpg
processing image camera_cal/calibration12.jpg
processing image camera_cal/calibration13.jpg
processing image camera_cal/calibration14.jpg
processing image camera_cal/calibration15.jpg
processing image camera_cal/calibration16.jpg
processing image camera_cal/calibration17.jpg
processing image camera_cal/calibration18.jpg
processing image camera_cal/calibration19.jpg
processing image camera_cal/calibration2.jpg
processing image camera_cal/calibration20.jpg
processing image camera_cal/calibration3.jpg
processing image camera_cal/calibration4.jpg
No corner found
processing image camera_cal/calibration5.jpg
No corner found
processing image camera_cal/calibration6.jpg
processing image camera_cal/calibration7.jpg
processing image camera_cal/calibration8.jpg
processing image camera_cal/calibration9.jpg
Prcoess camera cal image done


In [133]:

def cal_undistort(img, mtx, dist):
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist



def display_image(img1, img2, title1='Original', title2='Processed', format1='rgb', format2='rgb'):
    if format1 == 'bgr':
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    if format2 == 'bgr':
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img1)
    ax1.set_title(title1, fontsize=30)
    ax2.imshow(img2)
    ax2.set_title(title2, fontsize=30)
    plt.show()
    


In [134]:
def color_sobel(img, s_thresh=(170, 220), sx_thresh=(20, 100), img_format='bgr'):
    # Convert to HSV color space and separate the V channel
    if img_format == 'bgr':
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    else:
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    combined_binary = np.zeros_like(gray)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return (color_binary, combined_binary)




## Persective Transformation

In [135]:
def get_perspective_matrix(img_size):
    bot_width = 0.76
    mid_width = 0.09
    height_pct = 0.62
    bot_pct = 0.935
    offset = img_size[0] * 0.25
    src = np.float32([
        [img_size[0]*(0.5 - mid_width/2), img_size[1] *height_pct ],
        [img_size[0]*(0.5 + mid_width/2), img_size[1] *height_pct ],
        [img_size[0]*(0.5 + bot_width/2), img_size[1] *bot_pct ],
        [img_size[0]*(0.5 - bot_width/2), img_size[1] *bot_pct ]
    ])

    dst = np.float32([
        [offset,0], 
        [img_size[0] - offset, 0], 
        [img_size[0] - offset, img_size[1]], 
        [offset, img_size[1]]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

## Apply undistort and threshould and wrap

In [136]:
def display_hisgram(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    plt.figure(figsize=(8, 3))
    plt.plot(histogram)
    plt.show()




In [137]:
def get_fitting_points(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    line_left = []
    for i in range(len(left_fitx)):
        line_left.append((left_fitx[i],ploty[i]))
    pts_left = np.int32([line_left])

    line_right = []
    for i in range(len(right_fitx)):
        line_right.append((right_fitx[i],ploty[i]))

    line_right.reverse()
    pts_right = np.int32([line_right])


    # print(line_left)
    ouput_just_plot = warped_img


    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    pts = np.hstack((pts_left, pts_right))

    return pts

In [ ]:
M, Minv = get_perspective_matrix(img_size)
with open(points_file, 'rb') as f:
    calibrated_data = pickle.load(file=f)
mtx = calibrated_data['mtx']
dist = calibrated_data['dist']
def process_image(img, mtx=mtx, dist=dist, debug=False):
    img_size = (img.shape[1], img.shape[0])
    undistort = cal_undistort(img, mtx, dist)
    color_binary, combined_binary = color_sobel(undistort)
    binary_warped = cv2.warpPerspective(combined_binary, M, img_size, flags=cv2.INTER_LINEAR)
    pts = get_fitting_points(binary_warped)
    lane_img_wraped = np.zeros_like(img)
    cv2.fillPoly(lane_img_wraped, pts, (0, 255, 0))
    lane_only = cv2.warpPerspective(lane_img_wraped, Minv, img_size, flags=cv2.INTER_LINEAR)
    copy_src_img = np.copy(img)
    processed = cv2.addWeighted(img, 1, lane_only, 0.3, 0)
    if debug:
        display_image(img,undistort, 'Original', 'Undistort')
        display_image(color_binary, combined_binary, 'Stacked thresholds', 'S channel + gradient thresholds')
        display_image(combined_binary, binary_warped, 'Binary', 'binary_warped')
        display_hisgram(binary_warped)
        display_image(lane_img_wraped, lane_only, 'Wrapped', 'Lane only')
        display_image(img, processed, 'Original', 'Lane plot')
    return processed

img = mpimg.imread('test_images/straight_lines2.jpg')


dst = process_image(img, mtx, dist)


In [ ]:
import imageio
from moviepy.editor import VideoFileClip
from IPython.display import HTML
output_video = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
output_clip = clip1.fl_image(process_image)
%time output_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


 92%|█████████▏| 1161/1261 [03:01<00:17,  5.78it/s]